In [1]:
import sys
sys.path.append('..')

In [2]:
from helper import load_dotenv
from file_interaction import get_blobs, copy_blob, upload_file, get_related_filepath
from pathlib import Path
import pandas as pd
import re
from tqdm.auto import tqdm

In [3]:
dotenv = load_dotenv()

# Daten zusammenstellen

In [4]:
all_blobs = get_blobs('labelstudio_moire_data')
all_files = list(Path('/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/10_labelstudio_data').glob('./*.jpg'))
blobs_to_replace = [b.split('/')[-1] for b in all_blobs if 'vps2400dpi150lpi' in b]
all_file_names = [f.name for f in all_files]

In [5]:
file_map = {}

for f in all_file_names:
    if f not in file_map:
        file_map[f] = {
            'local' : False,
            'azure' : False
        }
    
    file_map[f]['local'] = True
    file_map[f]['azure'] = f in blobs_to_replace

for f in blobs_to_replace:
    if f not in file_map:
        file_map[f] = {
            'local' : False,
            'azure' : False
        }

    file_map[f]['azure'] = True
    file_map[f]['local'] = f in all_file_names

In [6]:
df = pd.DataFrame.from_dict(
    [
        {
            'img' : key,
            'local' : file_map[key]['local'],
            'azure' : file_map[key]['azure']
        } for key in file_map
    ]
)

df = pd.merge(
    df,
    df.img.str.extract(r'(.+?)\.vps2400dpi17?50?lpi\.(.+)\.4c_\d+\.jpg').rename(columns={0:'job', 1:'pdf_name'}),
    how="left",
    left_index=True,
    right_index=True
)

# bereits erledigte Bilder einpflegen

In [7]:
ps_all_blobs = [b.split('/')[-1] for b in all_blobs if '.ps2400dpi150lpi.' in b]

In [8]:
df_ps = pd.DataFrame(
    [
        re.match(r'(.+?)\.ps2400dpi17?50?lpi\.(.+)\.4c_\d+\.jpg', blob).groups()
        for blob in ps_all_blobs
    ],
    columns=['job','pdf_name']
)
df_ps.loc[
    :,
    'already_transferred'
] = True

In [9]:
df = pd.merge(
    df,
    df_ps,
    how="left",
    on=['job','pdf_name']
)
df.already_transferred.fillna(False, inplace=True)

In [10]:
jobs_to_exclude = [
    '23-10-18_farbe'
    '23-10-19_farbe',
    '23-10-23_glattesRaster'
]

df = df.loc[df.job.isin(jobs_to_exclude) == False]

In [11]:
idx_to_transfer = df.loc[df.already_transferred == False].index

In [24]:
def get_matching_ps_file( row ):
    return get_related_filepath( row.job, 'ps2400dpi150lpi', f'{row.pdf_name}.4c_600.jpg' )

In [27]:
errors = []

for idx in tqdm(idx_to_transfer):
    row = df.loc[idx]
    
    res = get_matching_ps_file( row )

    if res == None:
        errors.append(row)
    else:
        filepath, _ = res
        target_path = f'labelstudio_moire_data/{ row.job }.ps2400dpi150lpi.{ row.pdf_name }.4c_600.jpg'

        copy_blob(
            filepath,
            target_path
        )

  0%|          | 0/1803 [00:00<?, ?it/s]

In [22]:
filepath

'data/147694/ps2400dpi150lpi/tg51_bg93_bpl_gc_oel_gas_2022_sn_rz.p1'

In [26]:
target_path

'labelstudio_moire_data/147694.ps2400dpi150lpi.tg51_bg93_bpl_gc_oel_gas_2022_sn_rz.p1.4c_600.jpg'

In [15]:
df.loc[
    (df.index.isin([e.name for e in errors])) &
    (df.job.isin(['23-10-18_farbe','23-10-03_Testformen','23-10-17_blur']) == False)
].job.value_counts()

23-10-19_farbe    3
149355            1
Name: job, dtype: int64